In [28]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(Metrics)
library(rsq)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_num_finale.Rdata")

df_num <- df_num_finale

# Feature selection

In [29]:
library(leaps)

Best_Subset <-
    regsubsets(degenzaPostTraOSP~.,
               data =df_num,
               nbest = 1,      # 1 best model for each number of predictors
               nvmax = NULL,    # NULL for no limit on number of variables
               force.in = NULL, force.out = NULL,
               method = "exhaustive")
summary_best_subset <- summary(Best_Subset)

which.max(summary_best_subset$adjr2)

[1] 16

In [30]:
tmp <- summary_best_subset$which[18,]

tmp

(Intercept)                      age            degenzaPreTra 
                    TRUE                    FALSE                     TRUE 
           ratioPaO2FiO2         platCountTranspl                 albumina 
                    TRUE                     TRUE                     TRUE 
         surgeryDuration IntraoperativTransfected             plasmaFresco 
                    TRUE                     TRUE                     TRUE 
             plasmaPools           doseFibrinogen         CRIOprecipitated 
                    TRUE                    FALSE                     TRUE 
        CristalloidiComp           Albumina50Comp   noradrenalineEndSurger 
                    TRUE                     TRUE                     TRUE 
        intermedLactates             coldIschemia            organDonorAge 
                    TRUE                    FALSE                     TRUE 
        organDonorHeight organDonorDrawalSodiemia        organDonorGammaGT 
                    TRUE                     TRUE                     TRUE 
              realMELDNa 
                    TRUE

In [31]:
df_num_select <- subset(df_num, select= c(degenzaPreTra,ratioPaO2FiO2,platCountTranspl,albumina,surgeryDuration,IntraoperativTransfected,plasmaFresco,plasmaPools,CRIOprecipitated,CristalloidiComp,Albumina50Comp,noradrenalineEndSurger,intermedLactates,organDonorAge,organDonorHeight,organDonorDrawalSodiemia,organDonorGammaGT,realMELDNa,degenzaPostTraOSP))

In [32]:
dim(df_num_select)

[1] 1262   19

# Data partition

In [33]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_num$degenzaPostTraOSP, SplitRatio = 0.7)
train  <- subset(df_num, sample == TRUE)
test   <- subset(df_num, sample == FALSE)


In [34]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample_select <- sample.split(df_num_select$degenzaPostTraOSP, SplitRatio = 0.7)
train_select  <- subset(df_num_select, sample_select == TRUE)
test_select   <- subset(df_num_select, sample_select == FALSE)


# Model

In [35]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [36]:
lm <- train(degenzaPostTraOSP ~ ., data = train,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [37]:
lm_select <- train(degenzaPostTraOSP ~ ., data = train_select,
               method = "lm",
               trControl = ctrl)

+ Fold01.Rep1: intercept=TRUE 
- Fold01.Rep1: intercept=TRUE 
+ Fold02.Rep1: intercept=TRUE 
- Fold02.Rep1: intercept=TRUE 
+ Fold03.Rep1: intercept=TRUE 
- Fold03.Rep1: intercept=TRUE 
+ Fold04.Rep1: intercept=TRUE 
- Fold04.Rep1: intercept=TRUE 
+ Fold05.Rep1: intercept=TRUE 
- Fold05.Rep1: intercept=TRUE 
+ Fold06.Rep1: intercept=TRUE 
- Fold06.Rep1: intercept=TRUE 
+ Fold07.Rep1: intercept=TRUE 
- Fold07.Rep1: intercept=TRUE 
+ Fold08.Rep1: intercept=TRUE 
- Fold08.Rep1: intercept=TRUE 
+ Fold09.Rep1: intercept=TRUE 
- Fold09.Rep1: intercept=TRUE 
+ Fold10.Rep1: intercept=TRUE 
- Fold10.Rep1: intercept=TRUE 
Aggregating results
Fitting final model on full training set


In [38]:
lm

Linear Regression 

889 samples
 21 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 801, 800, 801, 799, 800, 799, ... 
Resampling results:

  RMSE       Rsquared   MAE     
  0.6500878  0.1770046  0.480168

Tuning parameter 'intercept' was held constant at a value of TRUE

In [39]:
lm_select

Linear Regression 

889 samples
 18 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 800, 799, 800, 801, 800, 801, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.6479107  0.1792889  0.4801659

Tuning parameter 'intercept' was held constant at a value of TRUE

In [40]:
# collect resamples
results <- resamples(list(lm=lm,  lm_select=lm_select))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: lm, lm_select 
Number of resamples: 10 

MAE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.4291547 0.4558700 0.4780966 0.4801680 0.5121765 0.5313748    0
lm_select 0.4266615 0.4632243 0.4720835 0.4801659 0.5019855 0.5473302    0

RMSE 
               Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.5405884 0.5966844 0.6581767 0.6500878 0.7034455 0.7358833    0
lm_select 0.5518515 0.6055436 0.6262579 0.6479107 0.7004188 0.7571134    0

Rsquared 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
lm        0.06126727 0.1172108 0.1681326 0.1770046 0.2325837 0.2900301    0
lm_select 0.10095498 0.1280467 0.1833077 0.1792889 0.2243216 0.2693748    0


In [41]:
vimp <- varImp(lm_select)
vimp
plot(vimp, cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.4)))

lm variable importance

                         Overall
surgeryDuration          100.000
realMELDNa                97.299
intermedLactates          76.883
noradrenalineEndSurger    60.614
plasmaPools               58.165
organDonorDrawalSodiemia  47.236
degenzaPreTra             43.880
ratioPaO2FiO2             41.515
plasmaFresco              38.254
Albumina50Comp            32.470
CRIOprecipitated          24.820
IntraoperativTransfected  23.083
organDonorGammaGT         20.002
albumina                  17.447
platCountTranspl          16.604
organDonorAge             13.384
CristalloidiComp           8.989
organDonorHeight           0.000

In [42]:
test_select$predicted <- predict(lm_select, newdata = test_select)
cor(test_select$degenzaPostTraOSP ,test_select$predicted)^2  #rsquared

[1] 0.255711

In [43]:
mae(test_select$degenzaPostTraOSP ,test_select$predicted)

[1] 0.4262604

In [44]:
rmse(test_select$degenzaPostTraOSP, test_select$predicted)

[1] 0.5576944

In [45]:
#predicted vs actual

ggplot(data = test_select, aes(x = predicted, y = degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = degenzaPostTraOSP), color="blue") +
  geom_line(aes(x = degenzaPostTraOSP,
                y = degenzaPostTraOSP), color = "black", 
            linetype = 2, size = 1)

`geom_smooth()` using method = 'loess' and formula 'y ~ x'



In [46]:
# residuals

ggplot(data = test_select, aes(x = predicted,
                      y = predicted - degenzaPostTraOSP)) +
  geom_point(alpha = 0.2, color = "blue") +
  geom_smooth(aes(x = predicted,
                  y = predicted - degenzaPostTraOSP),
              color="black")

`geom_smooth()` using method = 'loess' and formula 'y ~ x'

